In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
stroke_df=pd.read_csv("stroke.csv")

In [3]:
stroke_df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


## Changing everthing to numbers

Gender 1-> Male , 0-> Female

In [4]:
# stroke_df["gender"]

In [5]:
stroke_df["gender"] = (stroke_df["gender"]=="Male").astype(dtype=int)

In [6]:
stroke_df["gender"].value_counts()

0    2995
1    2115
Name: gender, dtype: int64

ever_married 1-> Yes , 0-> No

In [7]:
stroke_df["ever_married"].value_counts()

Yes    3353
No     1757
Name: ever_married, dtype: int64

In [8]:
stroke_df["ever_married"]=(stroke_df["ever_married"] == "Yes").astype(dtype=int)

In [9]:
stroke_df["ever_married"].value_counts()

1    3353
0    1757
Name: ever_married, dtype: int64


Residence_type 1-> Urban , 0-> Rural

In [10]:
stroke_df["Residence_type"] = (stroke_df["Residence_type"]=="Urban").astype(dtype=int)

In [11]:
stroke_df["Residence_type"].value_counts()

1    2596
0    2514
Name: Residence_type, dtype: int64

smoking_status

formerly smoked , smokes are 1

never smoked ,  Unknown    are 0       

In [12]:
stroke_df["smoking_status"].value_counts()

never smoked       1892
Unknown            1544
formerly smoked     885
smokes              789
Name: smoking_status, dtype: int64

In [13]:
# Smokers

ss=stroke_df.loc[(stroke_df["smoking_status"] == "smokes")]
fs=stroke_df.loc[(stroke_df["smoking_status"] == "formerly smoked")].copy()

smokers=np.array([ss,fs],dtype=object)

a1_df=pd.DataFrame(smokers[0])
a1_df["smoking_status"] = 1

c1_df=pd.DataFrame(smokers[1])
c1_df["smoking_status"] = 1


final_smokers=np.concatenate((a1_df.to_numpy(),c1_df.to_numpy()))


In [14]:
# Not Smokers

us=stroke_df.loc[(stroke_df["smoking_status"] == "Unknown")]
ns=stroke_df.loc[(stroke_df["smoking_status"] == "never smoked")]

not_smokers=np.array([us,ns],dtype=object)

a_df=pd.DataFrame(not_smokers[0])
a_df["smoking_status"] = 0

c_df=pd.DataFrame(not_smokers[1])
c_df["smoking_status"] = 0


final_not_smokers=np.concatenate((a_df.to_numpy(),c_df.to_numpy()))








In [15]:
# not_smokes_np=np.concatenate([ns.drop("work_type",axis=1).to_numpy(),us.drop("work_type",axis=1).to_numpy()]) # Converting to an numpy array of all smokes with 1

## Combining the numpy array to a single dataframe

In [16]:
# new_stroke_df=pd.DataFrame(np.concatenate((final_smokers,final_not_smokers)))
final_arr=np.concatenate((final_smokers,final_not_smokers))
new_stroke_df=pd.DataFrame(final_arr)
new_stroke_df.columns=stroke_df.columns

In [17]:
new_stroke_df["smoking_status"].value_counts()

0    3436
1    1674
Name: smoking_status, dtype: int64

In [18]:
new_stroke_df.dtypes

id                   object
gender               object
age                  object
hypertension         object
heart_disease        object
ever_married         object
work_type            object
Residence_type       object
avg_glucose_level    object
bmi                  object
smoking_status       object
stroke               object
dtype: object

In [19]:
 stroke_df=new_stroke_df.copy()   #Copying the Final output to our current dataframe

# Fixing nan in bmi

In [20]:
mean_bmi=stroke_df["bmi"].mean()
stroke_df["bmi"].fillna(mean_bmi,inplace=True)

In [21]:
stroke_df["bmi"].sort_values()

2124    10.3
2647    11.3
4032    11.5
1843    12.0
1908    12.3
        ... 
2109    66.8
3415    71.9
151     78.0
4763    92.0
2286    97.6
Name: bmi, Length: 5110, dtype: float64

 ## Shuffling the test and train data

In [22]:
stroke_df.dtypes

id                    object
gender                object
age                   object
hypertension          object
heart_disease         object
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level     object
bmi                  float64
smoking_status        object
stroke                object
dtype: object

In [23]:
from sklearn.model_selection import ShuffleSplit

x_stroke_df=stroke_df.drop("stroke",axis=1)
x_stroke_df=x_stroke_df.drop("id",axis=1)
x_stroke_df=x_stroke_df.drop("work_type",axis=1)


X=np.array(x_stroke_df)

y=np.array(stroke_df["stroke"]).astype('int')

sss=ShuffleSplit(n_splits=5,test_size=0.1,random_state=38)

In [24]:
for train_index,test_index in sss.split(X,y):
    print("Train : ", train_index , "Test : " , test_index)
    X_train,X_test=X[train_index] , X[test_index]
    y_train,y_test=y[train_index] , y[test_index]

Train :  [2282 1337  587 ... 4412 4149 3915] Test :  [5099 2213 5014 2074 4645 3524 3762 1454 3668 2301 2658 1104 1273 4260
 4706  125 3455 5086 3038 3551 4557 2865 4788 4931 2670 1515 4845 4540
 3674 3591 1455 1385  433 2854  527 2806  669 1607 3871 3849 1508 1053
 3178  188 2108 3715  171 2167 1620 3805  787 1754  549 2907 1056  696
 1780  405 2460 4274 4357 1005 4973 3320 3776 1340  197 4961 4079 3029
 3099 1224 3612 4780  468 4375  978 4025 2361 1580 3088  407 3954 1440
 1443  905 3409 3310 3476 2229 1673  258 2395 4512 3496 2993 1021 1376
 2383 4751 2925 2425 1670   28 4352   94 3248  145  858 2781 5016 1831
   62 3484 1028 3661 3819 3494 3087 1429  673 3014  127 3897 4197 2844
 4194 3446 3739 2681 3628 1256  515 3598 3713  176 4085 1834 2504 4055
 3227 3651 4681 3217 2676 1231 1037 1009 2403  839 3581 4602 2472 5070
 4521 3071  604 4290 1484 3263 1427 3659 2242 2753  860 3365 2272 4408
 1829 3922 2166 5003 4372 1501 3497 3976 4934 2946 2379   10  413 1343
 4161 3109  746 3884 197

In [25]:
# Using LogisticRegression

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

clf=LogisticRegression()
clf.fit(X_train,y_train)


KNeighborsClassifier(n_neighbors=10)

In [26]:
pred=clf.predict(X_test[344:400])
mse=mean_squared_error(y_test[344:400],pred)
print(np.sqrt(mse))

0.32732683535398854


In [27]:
print("Pred : \n" , pred , "\n Actual : " , y_test[344:400]) 

Pred : 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 
 Actual :  [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0]


In [29]:
from joblib import dump,load

dump(clf,"stroke_pred_model.joblib")
clf_load=load("stroke_pred_model.joblib")

In [30]:
pred_j=clf_load.predict(X_test)
mse_j=mean_squared_error(y_test,pred_j)
print(np.sqrt(mse_j))

0.2211869776019044


In [31]:
# 0.23822566551564467